In [2]:
!git clone https://github.com/franciscogarate/utils_ucm

fatal: destination path 'utils_ucm' already exists and is not an empty directory.


In [1]:
!pip install pyliferisk

In [8]:
import pandas as pd
import numpy as np

In [5]:
from pyliferisk import MortalityTable, lx
from pyliferisk.mortalitytables import PASEM2020_Decesos_M_1ord
mt = MortalityTable(qx=PASEM2020_Decesos_M_1ord)

In [10]:
edad = 50
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(mt.w -edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: mt.lx[x+1] if x <= mt.w else 0)
df.head(5)

,Fecha,edad,t,lx
0,2024-12-31,50,0,96712.874267
1,2025-12-31,51,1,96371.781936
2,2026-12-31,52,2,95995.343130
3,2027-12-31,53,3,95581.665965
4,2028-12-31,54,4,95128.984818


In [12]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df.head(5)

,Fecha,edad,t,lx,qx
0,2024-12-31,50,0,96712.874267,0.003527
1,2025-12-31,51,1,96371.781936,0.003892
2,2026-12-31,52,2,95995.343130,0.004277
3,2027-12-31,53,3,95581.665965,0.004681
4,2028-12-31,54,4,95128.984818,0.005103


In [13]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

df['capital'] = incr_capital(5000, df.t)
df.head(5)

,Fecha,edad,t,lx,qx,capital
0,2024-12-31,50,0,96712.874267,0.003527,5000.000000
1,2025-12-31,51,1,96371.781936,0.003892,5075.000000
2,2026-12-31,52,2,95995.343130,0.004277,5151.125000
3,2027-12-31,53,3,95581.665965,0.004681,5228.391875
4,2028-12-31,54,4,95128.984818,0.005103,5306.817753


In [14]:
df['pagos'] = df['capital'] * df['qx']
df.head(5)

,Fecha,edad,t,lx,qx,capital,pagos
0,2024-12-31,50,0,96712.874267,0.003527,5000.000000,17.634277
1,2025-12-31,51,1,96371.781936,0.003892,5075.000000,19.753595
2,2026-12-31,52,2,95995.343130,0.004277,5151.125000,22.033290
3,2027-12-31,53,3,95581.665965,0.004681,5228.391875,24.472382
4,2028-12-31,54,4,95128.984818,0.005103,5306.817753,27.080739


In [16]:
df['qx'].sum()

0.9999998756626617

In [18]:
i = 0.0218
df['factor_desc'] = df['t'].apply(lambda t : 1 / (1 + i) ** t )
df.head(2)

,Fecha,edad,t,lx,qx,capital,pagos,factor_desc
0,2024-12-31,50,0,96712.874267,0.003527,5000.0,17.634277,1.000000
1,2025-12-31,51,1,96371.781936,0.003892,5075.0,19.753595,0.978665


In [19]:
df.pagos @ df.factor_desc

4089.4779574323816

### Chequeo descuento lineal

In [20]:
!pip install numpy_financial

In [21]:
import numpy_financial as npf
npf.npv(i, df.pagos)

4089.4779574323816